In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.contrib.layers import fully_connected
from math import sqrt
from scipy.stats import norm
from scipy.stats import uniform
from sklearn.preprocessing import MinMaxScaler
import scipy

## Define some Variables

In [2]:
#Define a grid of strikes and maturities
T = np.array([2,2.5,3,3.5,4,4.5,5,5.5])
K = np.array([0.6,0.65,0.7,0.75,0.8,0.9,1.0,1.1])

num_input_parameters = len(T)*len(K)*4
num_output_parameters = len(T)*len(K)
learning_rate = 0.1
num_steps = 20
batch_size = 6
num_neurons = 15

#initial values
S0 = 1.0
V0 = 0.2
r = 0.15

#bounds for a,b,c,rho, make sure 2ab>c^2
bounds = np.array([[1,3],[0.1,0.6],[0,0.1],[0,1]])

## Next Batch Function

In [3]:
def corr_brownian_motion(n, T, dim, rho):
    dt = T/n

    dW1 = norm.rvs(size=(dim,n+1) , scale=sqrt(dt))
    dW2 = rho * dW1 + np.sqrt(1 - np.power(rho ,2)) * norm.rvs(size=(dim,n+1) , scale=sqrt(dt))
        
    W1 = np.cumsum(dW1, axis=-1)
    W2 = np.cumsum(dW2, axis=-1)
 
    return W1,W2

def euler_maruyama(mu,sigma,T,x0,W):
    dim = W.shape[0]
    n = W.shape[1]-1
    Y = np.zeros((dim,n+1))
    dt = T/n
    sqrt_dt = np.sqrt(dt)
    for l in range(dim):
        Y[l,0] = x0
        for i in range(n):
            Y[l,i+1] = Y[l,i] + np.multiply(mu(Y[l,i],l,i),dt) + sigma(Y[l,i],l,i)*sqrt_dt*(W[l,i+1]-W[l,i])
    
    return Y

def heston(a,b,c,T,W,Z,V0,S0):
    #assert(2*a*b > c*c) 
    
    def mu2(V,i,k):
        return np.multiply(a,(b-V))
    
    def sigma2(V,i,k):
        return np.multiply(c,np.sqrt(np.maximum(0.0,V)))
    
    V = euler_maruyama(mu2,sigma2,T,V0,Z)
    
    def mu1(S,i,k):
        return 0.0
    
    def sigma1(S,i,k):
        return np.multiply(np.sqrt(np.maximum(0.0,V[i,k])),S)
    
    S = euler_maruyama(mu1,sigma1,T,S0,W)
    
    return S,V

def reverse_transform_tensor(X_scaled):
    X = np.zeros(X_scaled.shape)
    for i in range(X_scaled.shape[-1]):
        X[:,:,:,i] = X_scaled[:,:,:,i]*(bounds[i][1]-bounds[i][0]) + bounds[i][0]
    return X

def next_batch_heston_EM_train(batch_size,bounds):
    X = np.zeros((batch_size,len(T),len(K),4))
    X_scaled = np.zeros((batch_size,len(T),len(K),4))
    y = np.zeros((batch_size,len(T)*len(K)))

    X_scaled = uniform.rvs(size=(batch_size,len(T),len(K),4))
    
    X = reverse_transform_tensor(X_scaled)

    n = 100
    dim = 10
        
    for i in range(batch_size):
        for j in range(len(T)):
            for k in range(len(K)):
                W,Z = corr_brownian_motion(n,T[j],dim,X[i,j,k,3])
                S,V = heston(X[i,j,k,0],X[i,j,k,1],X[i,j,k,2],T[j],W,Z,V0,S0)
                S_T = S[:,n]
        
                y[i,j*len(K)+k] = np.exp(-r*T[j])*np.mean(np.maximum(S_T-K[k],np.zeros(dim)))
    
    return X_scaled,y

## More Helper Functions

In [4]:
def init_weights(shape):
    init_random_dist = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(init_random_dist)

def init_bias(shape):
    init_bias_vals = tf.constant(0.1, shape=shape)
    return tf.Variable(init_bias_vals)

"""
Create a 2D convolution using builtin conv2d from TF. From those docs:
Computes a 2-D convolution given 4-D input and filter tensors.
Given an input tensor of shape [batch_size, len(T), len(K), #params] and a filter / kernel tensor of shape [filter_height, filter_width, in_channels, out_channels], this op performs the following:
Flattens the filter to a 2-D matrix with shape [filter_height * filter_width * in_channels, output_channels].
Extracts image patches from the input tensor to form a virtual tensor of shape [batch_size, len(T), len()K, filter_height * filter_width * #params].
For each patch, right-multiplies the filter matrix and the image patch vector.
"""
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

"""
Args:
  value: A 4-D `Tensor` with shape `[batch_size, len(T), len(K), #params]` and
    type `tf.float32`.
  ksize: A list of ints that has length >= 4.  The size of the window for
    each dimension of the input tensor.
  strides: A list of ints that has length >= 4.  The stride of the sliding
    window for each dimension of the input tensor.
  padding: A string, either `'VALID'` or `'SAME'`. 
"""
def max_pool_2by2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

def convolutional_layer(input_x, shape):
    W = init_weights(shape)
    b = init_bias([shape[3]])
    return tf.nn.relu(conv2d(input_x, W) + b)

def normal_full_layer(input_layer, size):
    input_size = int(input_layer.get_shape()[1])
    W = init_weights([input_size, size])
    b = init_bias([size])
    return tf.matmul(input_layer, W) + b

In [5]:
X = tf.placeholder(tf.float32,shape=[None,len(T),len(K),4])
y = tf.placeholder(tf.float32, shape=[None,len(T)*len(K)])

In [6]:
"""
3x3 Filter
4 `ìmages` as input
32 outputs
"""
convo_1 = convolutional_layer(X,shape=[3,3,4,32])
convo_1_pooling = max_pool_2by2(convo_1)

"""
3x3 Filter
32 inputs
64 outputs
"""
convo_2 = convolutional_layer(convo_1_pooling,shape=[3,3,32,32])
convo_2_pooling = max_pool_2by2(convo_2)


convo_2_flat = tf.reshape(convo_2_pooling,[-1,int(len(T)/2)*32])
full_layer_one = tf.nn.relu(normal_full_layer(convo_2_flat,1024))

outputs = fully_connected(full_layer_one, len(T)*len(K), activation_fn=None)

Instructions for updating:
Please use `layer.__call__` method instead.


In [7]:
#Loss Function
loss = tf.reduce_mean(tf.sqrt(tf.square(outputs - y)))  # MSE

#Optimizer
optimizer = tf.train.AdamOptimizer(learning_rate)
train = optimizer.minimize(loss)

init = tf.global_variables_initializer()

saver = tf.train.Saver()

In [12]:
with tf.Session() as sess:
    sess.run(init)
    
    for iteration in range(num_steps):
        
        X_batch,Y_batch = next_batch_heston_EM_train(batch_size,bounds)
   
        sess.run(train,feed_dict={X: X_batch, y: Y_batch})
        
        if iteration % 4 == 0:
            
            rmse = loss.eval(feed_dict={X: X_batch, y: Y_batch})
            print(iteration, "\tRMSE:", rmse)
    
    saver.save(sess, "./models/heston_cnn")

0 	RMSE: 452.9606
4 	RMSE: 2.1978002
8 	RMSE: 0.33170107
12 	RMSE: 0.118288405
16 	RMSE: 0.57613575


In [8]:
theta_true = [2.1,0.5,0.05,0.4]

In [9]:
def NNprediction(theta):
    x = np.zeros((1,len(T),len(K),len(theta)))
    for i in range(len(T)):
        for j in range(len(K)):
            x[0,i,j,:] = theta
    return sess.run(outputs,feed_dict={X: x})

def NNgradientpred(theta):
    x = np.zeros((1,len(T),len(K),len(theta)))
    for i in range(len(T)):
        for j in range(len(K)):
            x[0,i,j,:] = theta   
    grad = np.zeros((len(T)*len(K),len(theta)))
    
    delta = 0.000001
    for i in range(len(theta)):
        h = np.zeros(x.shape)
        h[0,:,:,i] = delta
        
        #two point gradient
        #grad[i] = (sess.run(outputs,feed_dict={X: x+h}) - sess.run(outputs,feed_dict={X: x-h}))/2/delta

        #four point gradient
        grad[:,i] = (-sess.run(outputs,feed_dict={X: x+2*h})+8*sess.run(outputs,feed_dict={X: x+h})-8*sess.run(outputs,feed_dict={X: x-h}) +sess.run(outputs,feed_dict={X: x-2*h}))/12/delta

    return grad

#input: theta in order a,b,c,rho
def CostFunc(theta):

    if theta[3]<=0:
        theta[3] = 0.01
    if theta[3]>=1:
        theta[3] = 0.99
    if theta[2]<=0:
        theta[2] = 0.1
    if 2*theta[0]*theta[1]<=theta[2]*theta[2]:
        theta[0] += theta[2]
        theta[1] += theta[2]
   
    heston_ = np.zeros(num_output_parameters)
    for i in range(len(T)):
        for j in range(len(K)):
            n = 500
            dim = 20
            W,Z = corr_brownian_motion(n,T[i],dim,theta_true[3])
            S,V = heston(theta_true[0],theta_true[1],theta_true[2],K[j],W,Z,V0,S0)
            S_T = S[:,n]
        
            heston_[i*len(K)+j] = np.exp(-r*T[i])*np.mean(np.maximum(S_T-K[j],np.zeros(dim)))
            
    return (NNprediction(theta)-heston_)[0]


def Jacobian(theta):

    if theta[3]<=0:
        theta[3] = 0.01
    if theta[3]>=1:
        theta[3] = 0.99
    if theta[2]<=0:
        theta[2] = 0.1
    if 2*theta[0]*theta[1]<=theta[2]*theta[2]:
        theta[0] += theta[2]
        theta[1] += theta[2]

    return NNgradientpred(theta)

In [ ]:
with tf.Session() as sess:                          
    #saver.restore(sess, "./models/heston_closed_nn") 
    saver.restore(sess, "./models/heston_cnn")   

    init = [bounds[0,0],bounds[1,0],bounds[2,0],bounds[3,0]]
    init_best = init
    
    cost_old = 1000000
    n = 3
    for a in range(n):
        print(a)
        init_new = [bounds[0,0],bounds[1,0],bounds[2,0],bounds[3,0]]
        init_new[0] += 1/(n+1)*(bounds[0,1]-bounds[0,0])
        for b in range(n):
            init_new[1] += 1/(n+1)*(bounds[1,1]-bounds[1,0])
            init_new[2] = bounds[2,0]
            init_new[3] = bounds[3,0]
            for c in range(n):
                init_new[2] += 1/(n+1)*(bounds[2,1]-bounds[2,0])
                init_new[3] = bounds[3,0]
                for d in range(n):
                    init_new[3] += 1/(n+1)*(bounds[3,1]-bounds[3,0])
                    
                    cost_new = CostFunc(init_new)
                    if np.linalg.norm(cost_new) < np.linalg.norm(cost_old):
                        init_best = init_new 
                    cost_old = cost_new
            
    init = init_best
print(init)

In [ ]:
with tf.Session() as sess:                          
    saver.restore(sess, "./models/heston_cnn")      
    
    init = [2,0.3,0.09,0.6]
    bnds = ([bounds[0,0],bounds[1,0],bounds[2,0],bounds[3,0]],[bounds[0,1],bounds[1,1],bounds[2,1],bounds[3,1]])
    X_batch,Y_batch = next_batch_heston_EM_train(batch_size,bounds)

    I=scipy.optimize.least_squares(CostFunc,init,Jacobian,bounds=bnds,gtol=1E-10,xtol=1E-10,verbose=1)

In [104]:
theta_nn = I.x
print("Predicted Theta: ",theta_nn)
print("True Theta: ",theta_true)

Predicted Theta:  [2.9119428  0.57358284 0.09559714 0.96477712]
True Theta:  [2.1, 0.5, 0.05, 0.4]


In [110]:
RMSE = 0
price_true = np.zeros((len(T),len(K)))
price_opt_params = np.zeros((len(T),len(K)))
theta_nn =  [1.1,0.3,0.05,0.2]
for i in range(len(T)):
    for j in range(len(K)):
        #price_true = heston_closed(theta_true[0],theta_true[1],theta_true[2],T[i],K[j],theta_true[3],V0,S0)
        n = 100
        dim = 20
        W,Z = corr_brownian_motion(n,T[i],dim,theta_true[3])
        S,V = heston(theta_true[0],theta_true[1],theta_true[2],T[i],W,Z,V0,S0)
        S_T = S[:,n]
        price_true[i,j] = np.exp(-r*T[i])*np.mean(np.maximum(S_T-K[j],np.zeros(dim)))
        
        #price_opt_params = heston_closed(Theta[i,j,0],Theta[i,j,1],Theta[i,j,2],T[i],K[j],Theta[i,j,3],V0,S0)
        W,Z = corr_brownian_motion(n,T[i],dim,theta_nn[3])
        S,V = heston(theta_nn[0],theta_nn[1],theta_nn[2],T[i],W,Z,V0,S0)
        S_T = S[:,n]
        price_opt_params[i,j] = np.exp(-r*T[i])*np.mean(np.maximum(S_T-K[j],np.zeros(dim)))
        RMSE += np.power(price_true[i,j]-price_opt_params[i,j],2)
RMSE = np.sqrt(RMSE/(len(T)*len(K)))
print("RMSE: ", RMSE)

RMSE:  0.034596460157678484
